## Importing packages

In [1]:
import random
import time
import pickle
import sys
import json
import re
import string
import collections
import codecs
from io import BytesIO

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import matplotlib as mpl

#from sklearn import feature_extraction

#import gensim
#from gensim import corpora
#from gensim.corpora import WikiCorpus
#from gensim.models import Word2Vec
#from gensim.models.word2vec import LineSentence

#import pyLDAvis
#import pyLDAvis.gensim

import nltk
from nltk.corpus import stopwords
from nltk.tag.sequential import ClassifierBasedTagger
#from nltk.stem.wordnet import WordNetLemmatizer
#from nltk.tokenize import word_tokenize

import pygraphviz
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout


from IPython.display import display, Image
from IPython.core.interactiveshell import InteractiveShell

pd.options.mode.chained_assignment = None  # default='warn'

## Developing a function to resolve abbreviations

In [2]:
abbreviations1 = pd.read_csv('./csv/fragebogen-abbreviations.csv', 
                             encoding="cp1250",
                             sep=',',
                             usecols=[0, 1], 
                             names=["abbreviated","expanded",])
abbreviations1.dropna(axis=0, inplace=True)
abbreviations1.head(10)

,abbreviated,expanded
0,.ähnl.,ähnlich
1,.Bed.,Bedeutung
2,.Bez.,Bezeichnung
4,.dgl.,dergleichen
5,.Füg.,Fügungen
6,.Komp.,Komposita
7,.Syn.,Synonyme
8,.Untersch.,Unterschied
9,.Vb.,Verba
10,.Volksüberlieferg.,Volksüberlieferung


In [3]:
abbreviations2 = pd.read_csv('./csv/fragebogen-abbreviations.csv', 
                             encoding="cp1250",
                             sep=',',
                             usecols=[2, 3], 
                             names=["abbreviated","expanded",])
abbreviations2.dropna(axis=0, inplace=True)
abbreviations2.head(10)

,abbreviated,expanded
3,.d.,der
20,a.,am
21,A.,Auge
22,ä.,ähnlich
53,Ang.,Angabe
78,B.,Bauer
79,b.,bei
147,d.,der
165,e.,ein
195,F.,Femininum


In [4]:
subst1 = {" " + row["abbreviated"]: " " + row["expanded"] + " " for index, row in abbreviations1.iterrows()}
subst2 = {" " + row["abbreviated"]: " " + row["expanded"] + " " for index, row in abbreviations2.iterrows()}

In [5]:
for key, value in subst1.items():
    print(key, value)
    
    if key == " .Vb.":      #show just the first few items
        break

 .ähnl.  ähnlich 
 .Bed.  Bedeutung 
 .Bez.  Bezeichnung 
 .dgl.  dergleichen 
 .Füg.  Fügungen 
 .Komp.  Komposita 
 .Syn.  Synonyme 
 .Untersch.  Unterschied 
 .Vb.  Verba 


In [6]:
for key, value in subst2.items():
    print(key, value)
    
    if key == " g.":      #show just the first few items
        break

 .d.  der 
 a.  am 
 A.  Auge 
 ä.  ähnlich 
 Ang.  Angabe 
 B.  Bauer 
 b.  bei 
 d.  der 
 e.  ein 
 F.  Femininum 
 f.  für 
 g.  gehen 


In [7]:
def expand_abbreviations(text):
    for key in subst1.keys():
        if key in text:
            text = text.replace(key, subst1[key])
            changes["abbreviation: "+key] = subst1[key]
        if key.lower() in text:
            text = text.replace(key.lower(), subst1[key])
            changes["abbreviation: "+key.lower()] = subst1[key]
    for key in subst2.keys():
        if key in text:
            text = text.replace(key, subst2[key])
            changes["abbreviation: "+key] = subst2[key]
        if key.lower() in text:
            text = text.replace(key.lower(), subst2[key])
            changes["abbreviation: "+key.lower()] = subst2[key]
    return(text)    

## Developing a POS and Lemmatizer for German

#### Getting POS - [training a classifier with the tiger sentence corpus](https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/)

In [8]:
#https://github.com/ptnplanet/NLTK-Contributions/blob/master/ClassifierBasedGermanTagger/ClassifierBasedGermanTagger.py

class ClassifierBasedGermanTagger(ClassifierBasedTagger):
    """A classifier based German part-of-speech tagger. It has an accuracy of
    96.09% after being trained on 90% of the German TIGER corpus. The tagger
    extends the NLTK ClassifierBasedTagger and implements a slightly modified
    feature detector.
    """

    def feature_detector(self, tokens, index, history):
        """Implementing a slightly modified feature detector.
        @param tokens: The tokens from the sentence to tag.
        @param index: The current token index to tag.
        @param history: The previous tagged tokens.
        """

        word = tokens[index]
        if index == 0: # At the beginning of the sentence
            prevword = prevprevword = None
            prevtag = prevprevtag = None
            #word = word.lower() # Lowercase at the beginning of sentence
        elif index == 1:
            prevword = tokens[index-1] # Note: no lowercase
            prevprevword = None
            prevtag = history[index-1]
            prevprevtag = None
        else:
            prevword = tokens[index-1]
            prevprevword = tokens[index-2]
            prevtag = history[index-1]
            prevprevtag = history[index-2]

        if re.match(r'[0-9]+([\.,][0-9]*)?|[0-9]*[\.,][0-9]+$', word):
            # Included "," as decimal point
            shape = 'number'
        elif re.compile(r'\W+$', re.UNICODE).match(word):
            # Included unicode flag
            shape = 'punct'
        elif re.match(r'([A-ZÄÖÜ]+[a-zäöüß]*-?)+$', word):
            # Included dash for dashed words and umlauts
            shape = 'upcase'
        elif re.match(r'[a-zäöüß]+', word):
            # Included umlauts
            shape = 'downcase'
        elif re.compile(r"\w+", re.UNICODE).match(word):
            # Included unicode flag
            shape = 'mixedcase'
        else:
            shape = 'other'

        features = {
            'prevtag': prevtag,
            'prevprevtag': prevprevtag,
            'word': word,
            'word.lower': word.lower(),
            'suffix3': word.lower()[-3:],
            #'suffix2': word.lower()[-2:],
            #'suffix1': word.lower()[-1:],
            'preffix1': word[:1], # included
            'prevprevword': prevprevword,
            'prevword': prevword,
            'prevtag+word': '%s+%s' % (prevtag, word),
            'prevprevtag+word': '%s+%s' % (prevprevtag, word),
            'prevword+word': '%s+%s' % (prevword, word),
            'shape': shape
            }
        return features

#### Training with all sentences

In [9]:
with open('./nlp/nltk_german_classifier_data.pickle', 'rb') as f:
    tiger_tagger = pickle.load(f)

In [10]:
tiger_tagger.tag(['Das'])[0]

('Das', 'ART')

#### Strategy 1 for lemma - [Reading the Lemmata from the tiger sentence corpus](https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/)

In [11]:
def read_lemmata_from_tiger_corpus(tiger_corpus_file, valid_cols_n=15, col_words=1, col_lemmata=2):
    lemmata_mapping = {}

    with open(tiger_corpus_file, encoding="utf8") as f:
        for line in f:
            parts = line.split()
            if len(parts) == valid_cols_n:
                w, lemma = parts[col_words], parts[col_lemmata]
                if w != lemma and w not in lemmata_mapping and not lemma.startswith('--'):
                    lemmata_mapping[w] = lemma

    return lemmata_mapping

lemmata_mapping = read_lemmata_from_tiger_corpus('./nlp/tiger_release_aug07.corrected.16012013.conll09')

In [12]:
lemmata_mapping['nahm']

'nehmen'

#### Strategy 2 for lemma - If we have the POS, we can use the [German Lemmatizer](https://github.com/WZBSocialScienceCenter/germalemma)  
('N...' (nouns), 'V...' (verbs), 'ADJ...' (adjectives), 'ADV...' (adverbs)


In [13]:
#https://github.com/WZBSocialScienceCenter/germalemma
#https://datascience.blog.wzb.eu/2016/07/13/accurate-part-of-speech-tagging-of-german-texts-with-nltk/
#https://datascience.blog.wzb.eu/2016/07/13/autocorrecting-misspelled-words-in-python-using-hunspell/
#sudo apt-get install libmysqlclient-dev
from germalemma import GermaLemma

# passing the word and the POS tag ("N" for noun)
lemmatizer = GermaLemma()

def germallema(word, pos):
    pos = tiger_tagger.tag([word])[0][1]
    lemma = lemmatizer.find_lemma(word, pos)
    return(lemma)

In [14]:
print(germallema("Arme", tiger_tagger.tag(["Arme"])[0][1]))
print(germallema("nahm", tiger_tagger.tag(["nahm"])[0][1]))
print(germallema("wegbegeben", tiger_tagger.tag(["wegbegeben"])[0][1]))

Arm
nehmen
wegbegeb


#### Strategy 4 - Using [hunspell](https://stackoverflow.com/questions/53029568/how-do-i-install-hunspell-python-package) for a fallback [option](https://datascience.blog.wzb.eu/2016/07/13/autocorrecting-misspelled-words-in-python-using-hunspell/)

In [15]:
#sudo apt-get install libhunspell-dev
#pip install hunspell
import hunspell

spellchecker = hunspell.HunSpell('./nlp/de_DE.dic','./nlp/de_DE.aff')
spellchecker_enc = spellchecker.get_dic_encoding()

In [16]:
def hunspell_lemma(word):
    try:
        w_lemma = spellchecker.stem(word)[-1].decode(spellchecker_enc)
        return(w_lemma)
    except:
        return(None)

In [17]:
hunspell_lemma('musste')

'musst'

#### Using all the resources together:

In [18]:
def robust_lemmatization(word):
    lemma = None
    try:
        pos = tiger_tagger.tag([word])[0][1]
        lemma = germallema(word, pos)
    except:
        pass
    #if not lemma:
    #    try:
    #        lemma = hunspell_lemma(word)
    #    except:
    #        pass
    if not lemma:
        try:
            lemma = lemmata_mapping[word]
        except:
            pass
    if lemma:
        return(lemma)
    else:
        return(word)
    
def lemmatize(word):
    lemma = robust_lemmatization(word)
    lemma = lemma.lower()
    if word.lower() != lemma.lower():
        changes["lemma: "+word] = lemma
        #print("Lemma: changed {} for {}".format(word, lemma))
    return(lemma)

In [19]:
print(robust_lemmatization("Arme"))
print(robust_lemmatization("wegbegeben"))

Arm
wegbegeb


## 1 - Fragebogen

#### Reading fragebogen information

In [20]:
fragebogen = pd.read_csv('./csv/fragebogen.csv', names=["id",
                                                        "nummer", 
                                                        "titel", 
                                                        "schlagwoerter", 
                                                        "erscheinungsjahr",
                                                        "autoren", 
                                                        "originaldaten",
                                                        "anmerkung",
                                                        "freigabe",
                                                        "checked",
                                                        "wordleiste",
                                                        "druck",
                                                        "online",
                                                        "publiziert",
                                                        "fragebogen_typ_id",])

fragebogen.drop(["id","schlagwoerter","erscheinungsjahr","autoren","originaldaten","anmerkung","freigabe",
                 "checked","wordleiste","druck","online","publiziert","fragebogen_typ_id",], inplace=True, axis=1)

fragebogen.set_index("nummer", drop=True, inplace=True)

In [21]:
fragebogen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 762 entries, 1 to ETir.28
Data columns (total 1 columns):
titel    762 non-null object
dtypes: object(1)
memory usage: 11.9+ KB


In [22]:
fragebogen.head(40)

,titel
nummer,
1,Fragebogen 1: Kopf (1)
2,Fragebogen 2: Die Osterwoche (1)
3,Fragebogen 3: Die Osterwoche (2)
4,Fragebogen 4: Kopf (2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...
6,Fragebogen 6: Menschl. Haar und Bart (= Kopf 3)
7,Fragebogen 7: Hochzeit (1)
8,Fragebogen 8: Hochzeit (2)
9,Fragebogen 9: Hochzeit (3)


#### Filtering the original fragebogen from the CSV file

In [23]:
fragebogen.tail()

,titel
nummer,
ETir.24,24. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.25,25. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.26,26. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.27,27. Ergänzungsfragebogen zum Tiroler Fragebuch
ETir.28,28. Ergänzungsfragebogen zum Tiroler Fragebuch


In [24]:
fragebogen[fragebogen.titel.str.startswith('Fragebogen')].tail()

,titel
nummer,
105,Fragebogen 105: Verkehr
106,Fragebogen 106: Wasser
107,Fragebogen 107: Gewerbe
108,Fragebogen 108: Gewerbe
109,Fragebogen 109: Gewerbe


In [25]:
fb_original = fragebogen[fragebogen.titel.str.startswith('Fragebogen')]

#### Extracting information from the fragebogen titel

In [26]:
regex1 = r'([Fragebon]+)\s{1}([0-9]+)[:]{1}([,A-ZÄÖÜa-zäöüß0-9.\s]+)[,\s]*([=\-\(\)\sA-ZÄÖÜa-zäöüß0-9]*)'

fb_original.titel.str.extract(regex1).head()
#fb_original.titel.str.extract(regex1).tail()

,0,1,2,3
nummer,,,,
1,Fragebogen,1,Kopf,(1)
2,Fragebogen,2,Die Osterwoche,(1)
3,Fragebogen,3,Die Osterwoche,(2)
4,Fragebogen,4,Kopf,(2)
5,Fragebogen,5,Zeit zwischen Ostern und Fronleichnam,


In [27]:
fb_original.loc[:,'fragebogen_num'] = fb_original.titel.str.extract(regex1)[1].str.strip()
fb_original.loc[:,'fragebogen_headwords'] = fb_original.titel.str.extract(regex1)[2].str.strip()
fb_original.loc[:,'fragebogen_series'] = fb_original.titel.str.extract(regex1)[3].str.strip()

In [28]:
fb_original.head()

,titel,fragebogen_num,fragebogen_headwords,fragebogen_series
nummer,,,,
1,Fragebogen 1: Kopf (1),1,Kopf,(1)
2,Fragebogen 2: Die Osterwoche (1),2,Die Osterwoche,(1)
3,Fragebogen 3: Die Osterwoche (2),3,Die Osterwoche,(2)
4,Fragebogen 4: Kopf (2),4,Kopf,(2)
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...,5,Zeit zwischen Ostern und Fronleichnam,


#### Expanding abbreviations, tokenizing, eliminating stopwords and lemmatizing

In [29]:
my_stopwords = set(stopwords.words('german')).union(set(["i", 
                                                         "ii", 
                                                         'e', 
                                                         'n',
                                                         'a',
                                                         'd',
                                                         'g',]))

In [30]:
#https://stackoverflow.com/questions/35231285/python-how-to-split-a-string-by-non-alpha-characters

def expand_tokenize_clean_lemmatize(input_text):
    output = re.sub(r'/',  ' ', input_text)
    output = re.sub(r'\(',  ' ', output)
    output = re.sub(r'\)',  ' ', output)
    output = expand_abbreviations(output)
    output = re.findall(r'\w+', output)
    output = [elem.strip(string.punctuation) for elem in output]
    output = [lemmatize(elem) for elem in output]
    output = [elem.lower() for elem in output if (elem.lower() not in my_stopwords)]
    output = [elem for elem in output if not elem.isnumeric()]
    output = list(set(output))
    #print(input_text, " ---> ", output)
    return output

In [31]:
changes = {}
fb_original['headwords_list'] = fb_original.fragebogen_headwords.apply(expand_tokenize_clean_lemmatize)

#for key, value in sorted(changes.items()):
#    print(key, "----> ", value)

In [32]:
fb_original.head(10)

,titel,fragebogen_num,fragebogen_headwords,fragebogen_series,headwords_list
nummer,,,,,
1,Fragebogen 1: Kopf (1),1,Kopf,(1),[kopf]
2,Fragebogen 2: Die Osterwoche (1),2,Die Osterwoche,(1),[osterwoche]
3,Fragebogen 3: Die Osterwoche (2),3,Die Osterwoche,(2),[osterwoche]
4,Fragebogen 4: Kopf (2),4,Kopf,(2),[kopf]
5,Fragebogen 5: Zeit zwischen Ostern und Fronlei...,5,Zeit zwischen Ostern und Fronleichnam,,"[ostern, fronleichnam, zeit]"
6,Fragebogen 6: Menschl. Haar und Bart (= Kopf 3),6,Menschl. Haar und Bart,(= Kopf 3),"[haar, menschl, bart]"
7,Fragebogen 7: Hochzeit (1),7,Hochzeit,(1),[hochzeit]
8,Fragebogen 8: Hochzeit (2),8,Hochzeit,(2),[hochzeit]
9,Fragebogen 9: Hochzeit (3),9,Hochzeit,(3),[hochzeit]


In [33]:
#print(fb_original.index)
print(fb_original.fragebogen_num.unique())
print()
print(fb_original.fragebogen_headwords.unique())
print()
print(set(fb_original.headwords_list.sum()))
print()
print(fb_original.fragebogen_series.unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28' '29' '30'
 '31' '32' '33' '34' '35' '36' '37' '38' '39' '40' '41' '42' '43' '44'
 '45' '46' '47' '48' '49' '50' '51' '52' '53' '54' '55' '56' '57' '58'
 '59' '60' '61' '62' '63' '64' '65' '66' '67' '68' '69' '70' '71' '72'
 '73' '74' '75' '76' '77' '78' '79' '80' '81' '82' '83' '84' '85' '86'
 '87' '88' '89' '90' '91' '92' '93' '94' '95' '96' '97' '98' '99' '100'
 '101' '102' '103' '104' '105' '106' '107' '108' '109']

['Kopf' 'Die Osterwoche' 'Zeit zwischen Ostern und Fronleichnam'
 'Menschl. Haar und Bart' 'Hochzeit' 'Nase, Ohr' 'Auge'
 'Besiedelung, Flur' 'Feld, Feldbestellung' 'Körper' 'Körperteile'
 'Hautkrankheiten' 'Brotbacken' 'Weißgebäck' 'Gliedmaßen, Arm' 'Arm'
 'Hand' 'Finger' 'Schneiderei' 'Kleidung' 'Bewegung' 'Farbe'
 'Zeit zwischen Mittsommer und Neujahr' 'Tod'
 'Gruß, Wunsch, Bitte und Dank' 'Jagd' 'Wald und Forstwirtschaft' 'Tiere'
 'Kind

In [34]:
fb_original.groupby("fragebogen_headwords")["fragebogen_num"].apply(lambda group_series: group_series.tolist()).reset_index().head(30)

,fragebogen_headwords,fragebogen_num
0,1. Erdoberfläche,[104]
1,Arm,[33]
2,Auge,[13]
3,"Besiedelung, Flur","[14, 15]"
4,Bewegung,"[42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]"
5,Bierbrauerei,[101]
6,"Blutsverwandtschaft, Altersstufen",[87]
7,Brotbacken,"[28, 29, 30]"
8,Das Pferd,[76]
9,Der Bauernhof,"[92, 93]"


In [35]:
fb_original.to_pickle('./fb_original.pickle')

## 2 - Frage

#### Reading frage information

In [36]:
frage = pd.read_csv('./csv/frage.csv', names=["id", "fragebogen_num","frage_num", "original_frage", "kurzfrage", "originaldaten"])

In [37]:
frage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24382 entries, 0 to 24381
Data columns (total 6 columns):
id                24382 non-null int64
fragebogen_num    24382 non-null int64
frage_num         24382 non-null object
original_frage    24382 non-null object
kurzfrage         17413 non-null object
originaldaten     6528 non-null object
dtypes: int64(2), object(4)
memory usage: 1.1+ MB


#### Filtering the relevant columns and the original questionnaires

In [38]:
frage = frage[frage.fragebogen_num <= 109]
frage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16744 entries, 0 to 16743
Data columns (total 6 columns):
id                16744 non-null int64
fragebogen_num    16744 non-null int64
frage_num         16744 non-null object
original_frage    16744 non-null object
kurzfrage         16744 non-null object
originaldaten     0 non-null object
dtypes: int64(2), object(4)
memory usage: 915.7+ KB


In [39]:
frage.drop(["originaldaten"], inplace=True, axis=1)

In [40]:
frage.head()

,id,fragebogen_num,frage_num,original_frage,kurzfrage
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr."
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)"
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf"


In [41]:
frage.tail()

,id,fragebogen_num,frage_num,original_frage,kurzfrage
16739,16740,109,602,Gewerbe: (Gehalts-)Zulage,(Gehalts-)Zulage
16740,16741,109,603,"Gewerbe: Abzug v. Gehalt, Pl.",Abzug v. Gehalt; Pl.
16741,16742,109,604,"Gewerbe: sonstige heute noch lebende, bodenstä...",sonst.bodenständige Berufe
16742,16743,109,605,"Gewerbe: abgekommene, bodenständige Berufe, di...","ehem.bodenständige Berufe (Waffenschmied,*)"
16743,16744,109,606,"Gewerbe: bodenständige FN, die vermutl. auf ei...","bodenständige FamN, die vermutl. auf ehem. Ber..."


#### Extracting information from the frage "original_frage" field

In [42]:
#regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}([A-ZÄÖÜa-zäöüß0-9=,;:\-\(\)\.\?\s]+)'
#regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}([A-ZÄÖÜa-zäöüß0-9=,;:\/\-\(\)\.\?\s]+)'
regex2 = r'([A-ZÄÖÜa-zäöüß.\(\)\-\/]+)[:]{1}(.+)'

frage.original_frage.str.extract(regex2).head(10)

,0,1
0,Kopf,"Kopf, Haupt; auch scherzh./übertr."
1,Kopf,Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Ko...
2,Kopf,Kopf/Haupt (übertr.) in Vergl./Ra. (das ist e...
3,Kopf,"schöner, ebenmäßig gebauter Kopf (Christuskop..."
4,Kopf,"häßlicher, unebenmäßiger Kopf"
5,Kopf,großer Kopf; großkopfig
6,Kopf,Wasserkopf
7,Kopf,kleiner Kopf
8,Kopf,oben zugespitzter Kopf
9,Kopf,langer Kopf


In [43]:
frage['main_theme'] = frage.original_frage.str.extract(regex2)[0]
frage.dropna(subset=['main_theme'], inplace=True,)
frage['question_words'] = frage.original_frage.str.extract(regex2)[1]

In [44]:
frage.head(20)

,id,fragebogen_num,frage_num,original_frage,kurzfrage,main_theme,question_words
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr.",Kopf,"Kopf, Haupt; auch scherzh./übertr."
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*,Kopf,Kopf/Haupt (in urspr. Bed.) in Vergl./Ra. (Ko...
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*,Kopf,Kopf/Haupt (übertr.) in Vergl./Ra. (das ist e...
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)",Kopf,"schöner, ebenmäßig gebauter Kopf (Christuskop..."
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf",Kopf,"häßlicher, unebenmäßiger Kopf"
5,6,1,A5,Kopf: großer Kopf; großkopfig,großer Kopf; großkopfig,Kopf,großer Kopf; großkopfig
6,7,1,A5a,Kopf: Wasserkopf,Wasserkopf,Kopf,Wasserkopf
7,8,1,A6,Kopf: kleiner Kopf,kleiner Kopf,Kopf,kleiner Kopf
8,9,1,A7,Kopf: oben zugespitzter Kopf,oben zugespitzter Kopf,Kopf,oben zugespitzter Kopf
9,10,1,A8,Kopf: langer Kopf,langer Kopf,Kopf,langer Kopf


#### Expanding abbreviations, tokenizing, eliminating stopwords and lemmatizing¶

In [45]:
changes = {}
frage.loc[:,'main_theme'] = frage['main_theme'].apply(expand_tokenize_clean_lemmatize)

#for key, value in sorted(changes.items()):
#    print(key, "----> ", value)

In [46]:
changes = {}
frage['question_words'] = frage.question_words.apply(expand_tokenize_clean_lemmatize)

#for key, value in sorted(changes.items()):
#    print(key, "----> ", value)

Adding the main theme to the list of question words

In [47]:
frage['question_words'] = frage['main_theme'] + frage['question_words']
frage['question_words'] = frage.question_words.apply(lambda x:list(set(x)))

Separating syntactic and semantic words in two columns 

In [48]:
syntactic = ['komposita', 
             'plural', 
             'deminutiv', 
             'femininum', 
             'adjektiv', 
             'synonyme', 
             'redensarten',
             'beispiel',
             'bedeutung',
             'substantiva', 
             'verb',
             'verba',
             'konjunktiv', 
             'dativ',
             'akkusativ',
             'singular',
             'ableitungen',
             #'name', 
             'bestimmungswort', 
             'grundwort', 
             'partizip', 
             'perfekt',
             'ab',
             'an',
             'et',
             'en', 
             'wendungen', 
             'ausdrücke', 
             'bezeichnung', 
             'wendungen', 
             'übertragen',
             'füg', 
             'aussprache',
             'ausdruck',
             'fügungen',
             'volkskundliches',
             'angabe',
             'spruch',
             'scherzhaft',
             'volkstümlich',
             'beschreibung',
             'ersatzwörter',
             'benennung',
             'umschreibungen',
             'benennungen',
             ]

def filter_words(list_of_words):
    list1 = [w for w in list_of_words if w not in syntactic]
    list2 = [w for w in list_of_words if w in syntactic]
    return list1, list2

In [49]:
frage['semantic_words'] = frage['question_words'].apply(lambda x: filter_words(x)[0])
frage['syntactic_words'] = frage['question_words'].apply(lambda x: filter_words(x)[1])

In [50]:
frage.head(100)
#frage.info()

,id,fragebogen_num,frage_num,original_frage,kurzfrage,main_theme,question_words,semantic_words,syntactic_words
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr.",[kopf],"[haupt, übertragen, kopf, scherzhaft]","[haupt, kopf]","[übertragen, scherzhaft]"
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*,[kopf],"[bedeutung, möchten, haupt, vergleich, kopf, u...","[möchten, haupt, vergleich, kopf, ursprünglich...","[bedeutung, redensarten]"
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*,[kopf],"[übertragen, offen, fein, kopf, vergleich, red...","[offen, fein, kopf, vergleich, jemand, verdreh...","[übertragen, redensarten]"
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)",[kopf],"[tituskopf, ebenmäßig, schön, christuskopf, ge...","[tituskopf, ebenmäßig, schön, christuskopf, ge...",[]
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf",[kopf],"[unebenmäßig, häßlich, kopf]","[unebenmäßig, häßlich, kopf]",[]
...,...,...,...,...,...,...,...,...,...
95,96,1,G25,Gesicht: Mitesser,Mitesser (im Gesicht),[gesicht],"[mitesser, gesicht]","[mitesser, gesicht]",[]
96,97,1,G26,"Gesicht: Gesichtsrose, Rotlauf und andere Erkr...","Gesichts-Erkrankgen (Gesichtsrose, Rotlauf)",[gesicht],"[erkrankung, rotlauf, gesichtsrose, gesicht]","[erkrankung, rotlauf, gesichtsrose, gesicht]",[]
97,98,1,H1,"Gesicht: Mund, Maul; Füg./Ra.",Mund/Maul; Füg./Ra.,[gesicht],"[maul, fügungen, mund, gesicht, redensarten]","[maul, mund, gesicht]","[fügungen, redensarten]"
98,99,1,H2,"Gesicht: großes, breites Maul (Adj.!), Mensch ...","großer, breiter Mund; entspr.Adj./Mensch; Ra.",[gesicht],"[maul, adjektiv, groß, mensch, breit, gesicht,...","[maul, groß, mensch, breit, gesicht, bildliche]","[adjektiv, wendungen]"


#### Examining the common concepts

In [51]:
concepts = []
for entry in frage.syntactic_words.values:  
    for concept in entry:
        concepts.append(concept)
#concepts = [c for c in concepts if c not in german_stopwords]
fdconcepts = collections.Counter(concepts)
fdconcepts.most_common(200)

[('komposita', 1661),
 ('redensarten', 1491),
 ('beispiel', 967),
 ('bedeutung', 866),
 ('plural', 697),
 ('deminutiv', 478),
 ('femininum', 414),
 ('ausdrücke', 401),
 ('adjektiv', 282),
 ('bezeichnung', 270),
 ('verba', 263),
 ('ableitungen', 218),
 ('synonyme', 198),
 ('wendungen', 189),
 ('ab', 183),
 ('übertragen', 159),
 ('füg', 110),
 ('aussprache', 90),
 ('grundwort', 87),
 ('ausdruck', 83),
 ('fügungen', 82),
 ('volkskundliches', 77),
 ('substantiva', 74),
 ('angabe', 73),
 ('spruch', 69),
 ('bestimmungswort', 68),
 ('scherzhaft', 66),
 ('volkstümlich', 62),
 ('beschreibung', 54),
 ('ersatzwörter', 50),
 ('benennung', 49),
 ('umschreibungen', 47),
 ('et', 47),
 ('benennungen', 34),
 ('en', 32),
 ('partizip', 31),
 ('perfekt', 30),
 ('singular', 29),
 ('verb', 12),
 ('akkusativ', 12),
 ('dativ', 4),
 ('konjunktiv', 4)]

In [52]:
concepts = []
for entry in frage.semantic_words.values:  
    for concept in entry:
        concepts.append(concept)
#concepts = [c for c in concepts if c not in german_stopwords]
fdconcepts = collections.Counter(concepts)
fdconcepts.most_common(500)

[('gewerbe', 609),
 ('körper', 479),
 ('besonderer', 380),
 ('wild', 343),
 ('feld', 321),
 ('wer', 298),
 ('gehen', 289),
 ('kopf', 274),
 ('jagd', 261),
 ('waldbaum', 256),
 ('ähnlich', 243),
 ('kommen', 234),
 ('name', 232),
 ('schneiderei', 214),
 ('teil', 204),
 ('bauer', 204),
 ('sonstig', 189),
 ('form', 180),
 ('auge', 176),
 ('baum', 173),
 ('gut', 171),
 ('eisenbahn', 168),
 ('holz', 166),
 ('üblich', 162),
 ('kind', 160),
 ('heißen', 155),
 ('art', 150),
 ('hautkrankh', 144),
 ('mensch', 141),
 ('brot', 141),
 ('vogel', 141),
 ('pferd', 138),
 ('wetter', 135),
 ('haut', 135),
 ('geschlecht', 133),
 ('lassen', 132),
 ('wagenteil', 128),
 ('hand', 127),
 ('geben', 125),
 ('reden', 124),
 ('klein', 123),
 ('gewässer', 123),
 ('groß', 122),
 ('sei', 122),
 ('rind', 120),
 ('bräuche', 118),
 ('wann', 117),
 ('fahren', 114),
 ('volksglaube', 113),
 ('stelle', 112),
 ('dergleichen', 112),
 ('schiffahrt', 110),
 ('haar', 107),
 ('verwendung', 107),
 ('führen', 104),
 ('fisch', 104),

In [53]:
anomalie = 'ab'
#anomalie = 'n'
#anomalie = 'e'
#anomalie = 'ra'
#anomalie = 'b'
#anomalie = 'komp'

for elem in frage[frage.question_words.apply(lambda x: anomalie in x)][['original_frage','question_words']][0:10].values:
    print(elem)
    print()

['Bart: Abbrennen d. Bartes, ab-, angebrannter Bart'
 list(['bartes', 'abbrennen', 'angebrannt', 'bart', 'ab'])]

['Feldbearbeitg.: Komp. mit bauen (in der Grundbedeutung "Feld bauen"), wie: an-, ab-, abhinbauen, ausbauen (mit dem Bau zu Ende sein), draufbauen (irgend eine Frucht auf ein Grundstück), überbauen (in fremdes Gebiet bauen), unterbauen (tief bauen), zubauen '
 list(['grundbedeutung', 'irgend', 'überbau', 'sei', 'ende', 'fremd', 'komposita', 'draufbauen', 'feldbearbeitg', 'frucht', 'feld', 'ab', 'zubau', 'bauen', 'bau', 'ausbau', 'unterbau', 'tief', 'abhinbau', 'gebiet', 'grundstück'])]

["Feld/Düngung: Dünger/Mist verkaufen; wie wird dabei gemessen? nach d. Fuhre, fuhrenweise? nach 'Krippen'?; gibt d. Bauer Mist gerne ab?"
 list(['messen', 'dünger', 'bauer', 'krippen', 'mist', 'verkaufen', 'geben', 'fuhrenweis', 'gerne', 'düngung', 'feld', 'ab', 'fuhre', 'dabei'])]

['Feld/pflügen: Rain machen, rainen, ab-, an-, verrainen (Bed.!)'
 list(['pflügen', 'bedeutung', 'rain', 'ver

#### (Sanity checking)

In [54]:
expression = "Feld/säen: Vb.Komp. mit säen, wie: an-, aus-, ein-, hin-, hinzu-, nach-, be-, versäen u.a.; überall Bed.Ang., auch übertr. Bed., z.B. er hat das ganze Geld versät/ angesät, d.h. verstreut, verloren"
print(re.findall(regex2, expression))
expand_tokenize_clean_lemmatize(re.findall(regex2, expression)[0][1])

[('Feld/säen', ' Vb.Komp. mit säen, wie: an-, aus-, ein-, hin-, hinzu-, nach-, be-, versäen u.a.; überall Bed.Ang., auch übertr. Bed., z.B. er hat das ganze Geld versät/ angesät, d.h. verstreut, verloren')]


['verba',
 'angabe',
 'hinzu',
 'ganz',
 'geld',
 'angesät',
 'be',
 'bedeutung',
 'übertragen',
 'versäen',
 'beispiel',
 'heißen',
 'verlieren',
 'überall',
 'verstreut',
 'komp',
 'versät',
 'säen']

### Using [Patern.de](https://www.clips.uantwerpen.be/pages/pattern-de)

In [55]:
import pattern.de as pde

In [56]:
print(pde.gender('Katze'))
print(pde.article('Katze', pde.DEFINITE, gender=pde.FEMALE, role=pde.OBJECT))
print(pde.singularize('Katzen'))
print(pde.pluralize('Katze'))
print(pde.conjugate('war', pde.INFINITIVE))
print(pde.conjugate('war', pde.PRESENT, 1, pde.SG, mood=pde.SUBJUNCTIVE))
print(pde.predicative('neugierige')) 
print(pde.attributive('neugierig', gender=pde.FEMALE))
print(pde.attributive('neugierig', gender=pde.FEMALE, role=pde.OBJECT))
print(pde.attributive('neugierig', gender=pde.FEMALE, role=pde.INDIRECT, article="die"))

f
die
Katze
Katzen
sein
sei
neugierig
neugierige
neugierige
neugierigen


In [57]:
s = pde.parse('Die böse Katze liegt auf der Matte.', tagset="STTS")
for sentence in s.split():
    print(sentence)

[['Die', 'ARTDEF', 'O', 'O'], ['böse', 'ADJA', 'O', 'O'], ['Katze', 'NN', 'B-NP', 'O'], ['liegt', 'VVFIN', 'B-PP', 'B-PNP'], ['auf', 'APPR', 'B-NP', 'I-PNP'], ['der', 'ARTDEF', 'O', 'O'], ['Matte', 'NN', 'B-NP', 'O'], ['.', 'S', 'O', 'O']]


In [58]:
s = pde.parse('Die böse Katze liegt auf der weichen Matte.')
for sentence in s.split():
    print(sentence)

[['Die', 'DT', 'B-NP', 'O'], ['böse', 'JJ', 'I-NP', 'O'], ['Katze', 'NN', 'I-NP', 'O'], ['liegt', 'VB', 'B-VP', 'O'], ['auf', 'IN', 'B-PP', 'B-PNP'], ['der', 'DT', 'B-NP', 'I-PNP'], ['weichen', 'PRP$', 'I-NP', 'I-PNP'], ['Matte', 'NN', 'I-NP', 'I-PNP'], ['.', '.', 'O', 'O']]


Dividing the concepts by gramatical classes

In [59]:
def onlynouns(l):
    nouns = [w for w in l if pde.parse(w).split('/')[1] == 'NN' or pde.parse(w.title()).split('/')[1] == 'NN']
    return(nouns)

def onlyverbs(l):
    verbs = [w for w in l if pde.parse(w).split('/')[1] == 'VB']
    return(verbs)

def onlyadject(l):
    adjects = [w for w in l if pde.parse(w).split('/')[1] == 'JJ']
    return(adjects)

In [60]:
frage_POS = pd.concat([frage.semantic_words.apply(onlynouns),
                       frage.semantic_words.apply(onlyverbs),
                       frage.semantic_words.apply(onlyadject)], axis=1)

frage_POS.columns = ['NN','VB', 'ADJ']
frage_POS.head(30)

,NN,VB,ADJ
0,"[haupt, kopf]",[],[]
1,"[haupt, vergleich, kopf, zerspringen]","[möchten, stehen]",[ursprünglich]
2,"[kopf, vergleich, verdrehen, haupt]",[],"[offen, fein]"
3,"[tituskopf, christuskopf, kopf]",[],"[ebenmäßig, schön]"
4,[kopf],[],"[unebenmäßig, häßlich]"
5,[kopf],[],"[großkopfig, groß]"
6,"[kopf, wasserkopf]",[],[]
7,"[klein, kopf]",[],[klein]
8,"[kopf, zugespitzt]",[],[]
9,[kopf],[],[lang]


In [61]:
frage_complete = pd.concat([frage, frage_POS], axis=1)
frage_complete.head()

,id,fragebogen_num,frage_num,original_frage,kurzfrage,main_theme,question_words,semantic_words,syntactic_words,NN,VB,ADJ
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr.",[kopf],"[haupt, übertragen, kopf, scherzhaft]","[haupt, kopf]","[übertragen, scherzhaft]","[haupt, kopf]",[],[]
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*,[kopf],"[bedeutung, möchten, haupt, vergleich, kopf, u...","[möchten, haupt, vergleich, kopf, ursprünglich...","[bedeutung, redensarten]","[haupt, vergleich, kopf, zerspringen]","[möchten, stehen]",[ursprünglich]
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*,[kopf],"[übertragen, offen, fein, kopf, vergleich, red...","[offen, fein, kopf, vergleich, jemand, verdreh...","[übertragen, redensarten]","[kopf, vergleich, verdrehen, haupt]",[],"[offen, fein]"
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)",[kopf],"[tituskopf, ebenmäßig, schön, christuskopf, ge...","[tituskopf, ebenmäßig, schön, christuskopf, ge...",[],"[tituskopf, christuskopf, kopf]",[],"[ebenmäßig, schön]"
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf",[kopf],"[unebenmäßig, häßlich, kopf]","[unebenmäßig, häßlich, kopf]",[],[kopf],[],"[unebenmäßig, häßlich]"


In [62]:
for entry in frage_complete.columns:
    frage_complete[entry] = frage_complete[entry].apply(lambda x:x if x != [] else None)

In [63]:
frage_complete.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16738 entries, 0 to 16743
Data columns (total 12 columns):
id                 16738 non-null int64
fragebogen_num     16738 non-null int64
frage_num          16738 non-null object
original_frage     16738 non-null object
kurzfrage          16738 non-null object
main_theme         16722 non-null object
question_words     16738 non-null object
semantic_words     16738 non-null object
syntactic_words    5598 non-null object
NN                 16649 non-null object
VB                 5930 non-null object
ADJ                6712 non-null object
dtypes: int64(2), object(10)
memory usage: 1.7+ MB


In [64]:
frage_complete.to_pickle('./frage.pickle')

In [65]:
frage_complete.to_csv('./frage.csv', sep='|')

In [66]:
frage_complete = pd.read_pickle('./frage.pickle')

### Classifying the questions

In [76]:
frage_complete[['id', 'fragebogen_num', 'frage_num', 'original_frage', 'kurzfrage',]]

,id,fragebogen_num,frage_num,original_frage,kurzfrage
0,1,1,A1,"Kopf: Kopf, Haupt; auch scherzh./übertr.","Kopf, Haupt; auch scherzh./übertr."
1,2,1,A2,Kopf: Kopf/Haupt (in urspr. Bed.) in Vergl./Ra...,Kopf/Haupt (urspr.Bed.) in Vergl./Ra.*
2,3,1,A3,Kopf: Kopf/Haupt (übertr.) in Vergl./Ra. (das ...,Kopf/Haupt (übertr.) in Vergl./Ra.*
3,4,1,A4,"Kopf: schöner, ebenmäßig gebauter Kopf (Christ...","schöner, ebenmäßiger Kopf (Christus-, Titusk.)"
4,5,1,A4a,"Kopf: häßlicher, unebenmäßiger Kopf","häßlicher, unebenmäßiger Kopf"
...,...,...,...,...,...
16739,16740,109,602,Gewerbe: (Gehalts-)Zulage,(Gehalts-)Zulage
16740,16741,109,603,"Gewerbe: Abzug v. Gehalt, Pl.",Abzug v. Gehalt; Pl.
16741,16742,109,604,"Gewerbe: sonstige heute noch lebende, bodenstä...",sonst.bodenständige Berufe
16742,16743,109,605,"Gewerbe: abgekommene, bodenständige Berufe, di...","ehem.bodenständige Berufe (Waffenschmied,*)"


In [86]:
frage_complete['frage_type'] = None

In [98]:
frage_complete['99' in frage_complete.frage_num].any()

KeyError: False

In [90]:
onomasiological_naming = ['Bez.',
                          '-bez.',
                          'Bezeichng.',
                          'wie nennt man',
                          'was bezeichnet',
                          'wie heißen',
                          'wie heißt',
                          'Benennung', 
                          'Benenng']

onomasiological_drawing = ['Bez.',
                          '-bez.',
                          'Bezeichng.',
                          'wie nennt man',
                          'was bezeichnet',
                          'wie heißen',
                          'wie heißt',
                          'Benennung', 
                          'Benenng']

In [91]:
def test_type(entry):
    guess = False
    pattern_list=onomasiological_naming
    for pattern in pattern_list:
        if pattern in entry:
            guess = True
    return guess

In [92]:
frage_complete.loc[frage_complete.original_frage.apply(test_type), 'frage_type'] = 'onomasiological'

In [93]:
frage_complete.frage_type.value_counts()

onomasiological    506
Name: frage_type, dtype: int64

### Using [Spacy](https://spacy.io/)

In [67]:
import spacy

In [68]:
# https://spacy.io/models
nlp = spacy.load("de_core_news_sm")
#nlp = spacy.load("de_core_news_md")

In [69]:
for l in frage.semantic_words.values[0:5]:
    for w in l:
        doc = nlp(w)
        for token in doc:
            print(token.text, 
                  token.lemma_, 
                  token.pos_, 
                  token.tag_, 
                  token.dep_,
                  token.shape_, 
                  token.is_alpha, 
                  token.is_stop)
    print()

haupt haupt ADJ ADJD ROOT xxxx True False
kopf kopf ADJ ADJD ROOT xxxx True False

möchten mögen VERB VMFIN ROOT xxxx True False
haupt haupt ADJ ADJD ROOT xxxx True False
vergleich vergleich ADJ ADJD ROOT xxxx True False
kopf kopf ADJ ADJD ROOT xxxx True False
ursprünglich ursprünglich ADJ ADJD ROOT xxxx True False
zerspringen zerspringen VERB VVINF ROOT xxxx True False
stehen stehen VERB VVINF ROOT xxxx True False

offen offen ADJ ADJD ROOT xxxx True True
fein fein ADJ ADJD ROOT xxxx True False
kopf kopf ADJ ADJD ROOT xxxx True False
vergleich vergleich ADJ ADJD ROOT xxxx True False
jemand jemand PRON PIS ROOT xxxx True True
verdrehen verdrehen VERB VVINF ROOT xxxx True False
haupt haupt ADJ ADJD ROOT xxxx True False

tituskopf tituskopf ADJ ADJD ROOT xxxx True False
ebenmäßig ebenmäßig ADJ ADJD ROOT xxxx True False
schön schön ADJ ADJD ROOT xxxx True False
christuskopf christuskopf X XY ROOT xxxx True False
gebaut bauen VERB VVPP ROOT xxxx True False
kopf kopf ADJ ADJD ROOT xxxx True

### Using [Watson Python API](https://github.com/watson-developer-cloud/python-sdk) 

### Translating Concepts

#### Using [GoogleTrans](https://pypi.org/project/googletrans/) 